# devlog 2024-08-28

_author: Trevor Johnson_

Demonstration of NPY and NPZ numpy ADRIOs.

In [1]:
# save a simple array to an .npy file and load it back using an adrio
from pathlib import Path

import numpy as np

from epymorph.adrio.numpy import NPY
from epymorph.error import DataResourceException

array = np.array([1, 2, 3])
np.save("./scratch/npy_test.npy", arr=array)

adrio = NPY(Path("./scratch/npy_test.npy"))
adrio.evaluate()

array([1, 2, 3])

In [2]:
# save multiple arrays to an .npz file and load them back individually using adrios
from epymorph.adrio.numpy import NPZ

array = np.array([4.4, 5.5, 6.6])
array2 = np.array(
    [(7.7, 8.8, 9.9)],
    dtype=[("data", np.float64), ("data2", np.float64), ("data3", np.float64)],
)
np.savez("./scratch/npz_test.npz", arr=array, arr2=array2)

# this adrio usees a slice to exclude the first element
adrio = NPZ(Path("./scratch/npz_test.npz"), "arr", np.s_[1:3,])
adrio2 = NPZ(Path("./scratch/npz_test.npz"), "arr2")


print(adrio.evaluate())
print(adrio2.evaluate())

[5.5 6.6]
[(7.7, 8.8, 9.9)]


In [3]:
# save and load a multidimensional array
array = np.array(
    [["This", "is", "an"], ["array", "of", "strings"], ["for", "testing", "."]]
)
np.savez("./scratch/npz_test2.npz", array)

# slice each axis of the array individually
adrio = NPZ(Path("./scratch/npz_test2.npz"), "arr_0", (slice(0, 2), slice(1, 3)))

adrio.evaluate()

array([['is', 'an'],
       ['of', 'strings']], dtype='<U7')

In [4]:
# single slice
adrio = NPZ(Path("./scratch/npz_test2.npz"), "arr_0", slice(0, None, 2))
adrio.evaluate()

array([['This', 'is', 'an'],
       ['for', 'testing', '.']], dtype='<U7')

In [5]:
# np.s_ support
adrio = NPZ(Path("./scratch/npz_test2.npz"), "arr_0", np.s_[0:2, ::2])
adrio.evaluate()

array([['This', 'an'],
       ['array', 'strings']], dtype='<U7')

In [6]:
# ellipsis support: note -- fewer indices than axes is actually allowable
adrio = NPZ(Path("./scratch/npz_test2.npz"), "arr_0", np.s_[...])
adrio.evaluate()

array([['This', 'is', 'an'],
       ['array', 'of', 'strings'],
       ['for', 'testing', '.']], dtype='<U7')

In [7]:
# np.index_exp support
adrio = NPZ(Path("./scratch/npz_test2.npz"), "arr_0", np.index_exp[...])
adrio.evaluate()

array([['This', 'is', 'an'],
       ['array', 'of', 'strings'],
       ['for', 'testing', '.']], dtype='<U7')

In [8]:
# ERROR: too many indices
try:
    adrio = NPZ(Path("./scratch/npz_test2.npz"), "arr_0", np.index_exp[1:3, 1:3, 1:3])
    adrio.evaluate()
except DataResourceException as e:
    print(repr(e))

DataResourceException('Specified array slice is invalid for the shape of this data.')


In [9]:
# ERROR: wrong file type
try:
    adrio = NPY(Path("./scratch/npz_test2.npz"))
except DataResourceException as e:
    print(repr(e))

DataResourceException('Incorrect file type. Only .npy files can be loaded through NPY ADRIOs.')
